In [170]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

tf.set_random_seed(777)

In [171]:
a = np.array(pd.read_csv("./heart.csv"))
a

array([[  0.,  63.,   1., ...,   0.,   2.,   1.],
       [  1.,  67.,   1., ...,   3.,   0.,   0.],
       [  2.,  67.,   1., ...,   2.,   1.,   0.],
       ...,
       [232.,  44.,   1., ...,   0.,   2.,   0.],
       [233.,  63.,   1., ...,   2.,   1.,   0.],
       [234.,  63.,   0., ...,   0.,   0.,   0.]])

In [172]:
scaler = MinMaxScaler(feature_range=(0,1))
for i in range(1,len(a[0,:])):
    a[:,i] = a[:,i]/np.mean(a[:,i])
a = scaler.fit_transform(a)

In [173]:
data = np.array(a)
x_data = data[:,1:-1]
y_data = data[:,-1]
'''
for i in range(len(y_data)):
    if y_data[i] == 'No':
        y_data[i] = 0.0
    else:
        y_data[i] = 1.0
'''
y_data = y_data.reshape(235,1)
x_data = np.array(x_data,dtype=np.float32)

In [174]:
b = np.array(pd.read_csv("./heart_test.csv"))
for i in range(1,len(b[0,:])):
    b[:,i] = b[:,i]/np.mean(b[:,i])
b = scaler.fit_transform(b)

x_test = b[:,1:-1]
y_test = b[:,-1]
'''
for i in range(len(y_data)):
    if y_data[i] == 'No':
        y_data[i] = 0.0
    else:
        y_data[i] = 1.0
'''
y_test = y_test.reshape(57,1)
x_test = np.array(x_test,dtype=np.float32)

In [175]:
num = 13
X = tf.placeholder(tf.float32,[None,num])
Y = tf.placeholder(tf.float32,[None,1])

In [176]:
W = tf.Variable(tf.random_normal([num,1]),name = 'weight')
b = tf.Variable(tf.random_normal([1]),name = 'bias')

In [177]:
hypothesis = tf.sigmoid(tf.matmul(X,W)+b)
#hypothesis = tf.nn.softmax(tf.matmul(X,W)+b)
cost = -tf.reduce_mean(Y*tf.log(hypothesis)+(1-Y)*tf.log(1-hypothesis))
#cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis)))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

predicted = tf.cast(hypothesis >0.5,dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),tf.float32))

In [178]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(20001):
        sess.run(optimizer, feed_dict= {X:x_data,Y:y_data})
        if step %2000 ==0:
            print(step, sess.run([cost],feed_dict={X:x_data,Y:y_data}))
    print(sess.run([accuracy],feed_dict={X:x_test,Y:y_test}))
    print(sess.run([accuracy],feed_dict={X:x_data,Y:y_data}))

0 [1.8267897]
2000 [0.36884573]
4000 [0.36287785]
6000 [0.36079323]
8000 [0.3598568]
10000 [0.35938904]
12000 [0.35913894]
14000 [0.35899925]
16000 [0.35891876]
18000 [0.35887158]
20000 [0.35884368]
[0.24561404]
[0.82978725]
